# Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations

## Author:M. Raissi, P. Perdikaris, G.E. Karniadakis

[github](https://github.com/maziarraissi/PINNs)

This piece of code is to predict the velocity fields and the pressure of a fluid system's flow status, based on Naiver-Stoke 2D equation,the results are demonstrated in **tensorflow 2 using the SSH data**. As the original code was written in tensorflow 1, variations were made to compat the latest version

In this work, we consider parametrized and nonlinear partial differential equations of the general form as

$u_t + N[u; λ] = 0, x ∈ \mathsf{\Omega}􏰀, t ∈ [0, T]$

where $u_t$ denotes the latent solution, $N[u; λ]$ is a nonlinear operator parameterized by λ, a weight variable

The current code is for the continuous time model, hence from the above we can define
$f :=u_t +N[u]$
by proceeding $u(t,x)$ to the neural network, we can get $f(t,x)$ as the result.

This network can be derived by applying the chain rule for differentiating compositions of functions using automatic differentiation, and has the same parameters as the network representing $u(t,x)$, albeit with different activation functions due to the action of the differential operator $N$. But both $u(t,x)$ and $f(t,x)$ can be operated by minimizing the mean squared error loss where:
$$MSE=MSE_u+MSE_f$$
$$MSE_U = \frac{1}{N_u}\sum_{i=1}^{N_u}\lvert u(t_u^i,x_u^i)-u^i \rvert^2$$
$$MSE_f = \frac{1}{N_f}\sum_{i=1}^{N_f}\lvert f(t_f^i,x_f^i) \rvert^2$$

As the $MSE_U$ represent the loss for the first neural net,a multi-layered neural network, with hyperbolic tangent as the activation function, $MSE_f$ is for the second neural net,a single layer neural network, with Naiver-Stokes function(see below) as the activation function. The $MSE_f$ is included to the total loss, this applys the Naiver-Stokes function by balancing the $MSE_U$ to improve the final loss $MSE=MSE_u+MSE_f$ computed


Where the ${t_u^i,x_u^i,u^i}$ denote the initial and boundary training data on $u(t,x)$ and $[t_f^i,x_f^i]_{i=1}^{N_f}$ specify the collocations points ${MSE}_u$ corresponds to the initial and boundary data while ${MSE}_f$ enforces the structure imposed by equation  at a finite set of collocation points.

In 2-D Naiver-Stoke equation, we can define that
$$u_t+λ_1(u*u_x +v*u_y)=-p_x+λ_2(u_{xx} +u_{yy})$$
$$v_t+λ_1(u*v_x +v*v_y)=-p_y+λ_2(v_{xx} +v_{yy})$$

hence we can derive $f(x,y,t)$ and $g(x,y,t)$, where g is simillar as f but it is the result for vertical flow v, 

$$f :=u_t +λ_1(u*u_x +v*u_y)+p_x −λ_2(u_{xx} +u_{yy})$$

$$g:=v_t +λ_1(u*v_x +v*v_y)+p_y −λ_2(v_{xx} +v_{yy})$$

On the equations $λ_1$ and $λ_2$ are the weighting parameter, unknown for now, solutions to the Navier–Stokes equations are searched in the set of divergence-free functions; i.e.,
$$u_x +v_y =0$$

This extra equation is the continuity equation for incompressible fluids that describes the conservation of mass of the fluid.
We make the assumption that$u=ψ_y, v=−ψ_x$ for latent function $ψ(t,x,y)$. 

Under this assumption, the continuity equation $u_x +v_y =0$ will be automatically satisfied.

Given the measurements {$t^i,x^i,y^i,u^i,v^i,p^i$} of the velocity field, what we need is the parameters λ as well as the pressure $p$, by taking the result the Naiver-Stoke parameter,[$f(x,y,t),g(x,y,t),ψ(x,y,t),p(x,y,t)$] and λ can be trained by minimizing the MSE loss

$$MSE = \frac{1}{N}\sum_{i=1}^{N}(\lvert{u(t^i,x^i,y^i)-u^i}\rvert+\lvert{v(t^i,x^i,y^i)-v^i}\rvert )+ \frac{1}{N}\sum_{i=1}^{N}(\lvert{f(t^i,x^i,y^i)^2}\rvert+\lvert{g(t^i,x^i,y^i)^2}\rvert )$$

The following are the the attributes in the code:

$x$:horizontal flow of fluid

$y$:vertical flow of fluid

$t$:time stamp

$u$:x component of the velocity field

$v$:y component of the velocity field

$p$:pressure

$f$:horizontal result of the neural network

$g$:vertical result of the neural network

In the neural network below, x,y,and t are the input, v,p

### Neural Networks

Artificial neural networks (ANNs), usually simply called neural networks (NNs), are computing systems vaguely inspired by the biological neural networks that constitute animal brains.

An neural network is based on a collection of artificial neurons, which creates the model simillarly to a brain. Each connection, like the synapses in a biological brain, can transmit a signal to other neurons. An artificial neuron that receives a signal then processes it and can signal neurons connected to it.
The "signal" at a connection is a real number, and the output of each neuron is computed by some non-linear function of the sum of its inputs. The connections are called edges.

Neurons and edges typically have a weight that adjusts as learning proceeds. The weight increases or decreases the strength of the signal at a connection, hence improves the final loss function. Neurons may have a threshold such that a signal is sent only if the aggregate signal crosses that threshold. Typically, neurons are aggregated into layers. Different layers may perform different transformations on their inputs. Signals travel from the first layer (the input layer), to the last layer (the output layer), possibly after traversing the layers multiple times, the size of the input layer and the output layer has to be the same size with the number of inputs and outputs.

Simplified view of a feedforward artificial neural network we'll be using later on


In the first neural net

**``xavier_init``** function is used to initialize the weight matrices with [``tf.random.truncated_normal``](https://www.tensorflow.org/api_docs/python/tf/random/truncated_normal) function generating random variable matrices

**``initialize_NN``** function is implemented to map the weights and biases into the shape and size of the layers and neurons applyed later on, but in this case, biases are all set to zeros, the weights are from the **``xavier_init``** function above.For instance if the layer has been set to [3,20,20,2], the output of the ``initialize_NN`` function will be weight matrices with in the shape of [3,20],[20,20] and [20,2].

In **``neural_net``** function a lambda function is applied to the neural net $H=2.0*(X - lb)/(ub - lb) - 1.0$, where $X$ is the wrap of all input variables, $lb$,$ub$ are the minimum and maximum variable in $X$, the activation function applied is hyperbolic tangent or $tanh$, inside the function is the lambda is updated for each layer $H=tanh(H*[weights]+[biases])$ where the weights and biases belong to the corresponding layer, H has the shape of ``[training size,neurons]``

At the output layer, the output function is $Y=H*[weights]+[biases]$, uses the previous lambda value $H$. The output $Y$ is the wrap up of $\psi$ and $p$

Creates the model class, and initialize the layers, parameters and placeholders, the **``xavier_init``** and **``initialize_NN``** are to construct the full structure of the weight and bias matrices, making them the same length and width with the layers setting. 

**``neural_net``** is a simple feedforward nueral network, with the $x$,$y$ and $t$ as the inputs, $ψ$ and $p$ are the outputs

**``net_NS``** is a second neural net, to proceed the Naiver-Stoke equation,it take $x$,$y$ and $t$ as the inputs, mutiplying devriatives based on equation [1] and [2]. The outputs are the $u$,$v$,$p$ prediction for further testing and verifaction

The **``training``** function in the original code, has applied Adam optimizer and scipy optimizer(with L-BFGS method), but the latter is no longer available for tensorflow 2, the simillar replacement in tfp is not very useful too, hence only the Adam optimizer is implemented for now. Adam optimizer would optimize the function, updating the lambda in [1] and [2], as well as the final loss value

Finally, the **``prediction``** function is for testing, it will apply the latest training function with the testing data, the predicted results are $u$, $v$ and $p$

First import packages

In [ ]:
import sys
sys.path.insert(0, '../../Utilities/')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import latex
import csv
import contextlib
from scipy.interpolate import griddata
import time
import meshio
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotting import newfig, savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import tensorflow_probability as tfp
import external_optimizer as opt

Set seeds for reproducibility

In [ ]:
np.random.seed(1234)
tf.random.set_seed(1234)
tf.compat.v1.disable_v2_behavior()

In [ ]:
class PhysicsInformedNN:       
    # Initialize the class 
    def __init__(self, x, y, t, u, v, layers):
        
        X = np.concatenate([x, y, t], 1)
        
        self.lb = X.min(0)
        self.ub = X.max(0)
                
        self.X = X
        
        self.x = X[:,0:1]
        self.y = X[:,1:2]
        self.t = X[:,2:3]
        
        self.u = u
        self.v = v
        
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)        
        
        # Initialize parameters
        self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        self.lambda_2 = tf.Variable([0.0], dtype=tf.float32)
        
        # tf placeholders and graph
        self.sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x_tf = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.x.shape[1]])
        self.y_tf = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.y.shape[1]])
        self.t_tf = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.t.shape[1]])
        
        self.u_tf = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.u.shape[1]])
        self.v_tf = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.v.shape[1]])
        #f_u_pred and f_v_pred corresponds to the f and g in the front
        self.u_pred, self.v_pred, self.p_pred, self.f_u_pred, self.f_v_pred = self.net_NS(self.x_tf, self.y_tf, self.t_tf)
        
        self.loss = tf.reduce_sum(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_sum(tf.square(self.v_tf - self.v_pred)) + \
                    tf.reduce_sum(tf.square(self.f_u_pred)) + \
                    tf.reduce_sum(tf.square(self.f_v_pred))                 
        self.lossu=tf.reduce_sum(tf.square(self.u_tf - self.u_pred)) + \
                   tf.reduce_sum(tf.square(self.v_tf - self.v_pred))
        self.lossf=tf.reduce_sum(tf.square(self.f_u_pred)) + \
                    tf.reduce_sum(tf.square(self.f_v_pred)) 
        
        self.optimizer = opt.ScipyOptimizerInterface(self.loss, 
                                                     method = 'L-BFGS-B', 
                                                     options = {'maxiter': 50000,
                                                                'maxfun': 50000,
                                                                'maxcor': 50,
                                                                'maxls': 50,
                                                                'ftol' : 1.0 * np.finfo(float).eps})        
        
        self.optimizer_Adam = tf.compat.v1.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        
        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        #Initialize the content of the weight matrix
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
    @tf.autograph.experimental.do_not_convert   
    def net_NS(self, x, y, t):
        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2
        psi_and_p = self.neural_net(tf.concat([x,y,t], 1), self.weights, self.biases)
        psi = psi_and_p[:,0:1]
        p = psi_and_p[:,1:2]
        u = tf.gradients(psi, y)[0]
        v = -tf.gradients(psi, x)[0]
        
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        
        v_t = tf.gradients(v, t)[0]
        v_x = tf.gradients(v, x)[0]
        v_y = tf.gradients(v, y)[0]
        
        v_xx = tf.gradients(v_x, x)[0]
        v_yy = tf.gradients(v_y, y)[0]
        
        p_x = tf.gradients(p, x)[0]
        p_y = tf.gradients(p, y)[0]

        f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy) 
        f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy) 
        
        return u, v, p, f_u, f_v
    
    def train(self, nIter): 
        finl=[]
        lu=[]
        lf=[]
        tf_dict = {self.x_tf: self.x, self.y_tf: self.y, self.t_tf: self.t,
                   self.u_tf: self.u, self.v_tf: self.v}        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            # Print
            if it % 10 == 0:
                
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                loss_value_u = self.sess.run(self.lossu,tf_dict)
                loss_value_f = self.sess.run(self.lossf,tf_dict)
                lambda_1_value = self.sess.run(self.lambda_1)
                lambda_2_value = self.sess.run(self.lambda_2)
                finl.append(loss_value)
                lu.append(loss_value_u)
                lf.append(loss_value_f)
                print('It: %d, Loss: %.3e, Loss_u:%.3e , Loss_f:%.3e , l1: %.5f, l2: %.5f, Time: %.2f' % 
                      (it, loss_value,loss_value_u,loss_value_f ,lambda_1_value, lambda_2_value, elapsed))
                start_time = time.time()
        plt.figure(1) 
        plt.plot(np.arange(0,nIter/10),finl)
        plt.figure(2)
        plt.plot(np.arange(0,nIter/10),lu)
        plt.figure(3)
        plt.plot(np.arange(0,nIter/10),lf) 
        with open('names.csv', 'w') as csvfile:
            fieldnames = ['finl','lu','lf']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for i in range(round(nIter/10)):
                writer.writerow({"finl":finl[i],'lu':lu[i],'lf':lf[i]})
    def callback(self, loss, lambda_1, lambda_2):
        print('Loss: %.3e, l1: %.3f, l2: %.5f' % (loss, lambda_1, lambda_2))  
        
    def predict(self, x_star, y_star, t_star):
        
        tf_dict = {self.x_tf: x_star, self.y_tf: y_star, self.t_tf: t_star}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        v_star = self.sess.run(self.v_pred, tf_dict)
        p_star = self.sess.run(self.p_pred, tf_dict)
        
        return u_star, v_star, p_star

Set up the training datasize N_train and layers

In [ ]:
if __name__ == "__main__": 
    N_train = 3000
    
    layers = [3, 40, 40, 40, 40, 40, 40, 40, 40, 40, 2] #11

### Training Stage for noiseless data
Import the data from file and process it for training

In [ ]:
    # Load Data
    # ——————————————————————————————————————————————————————————————————
    mesh = meshio.read(
    filename='/Users/howardxu/work/neuro-net/Physics-informed-neural-networks/main/Data/rec000068.vtu', 
    # string, os.PathLike, or a buffer/open file
      # optional if filename is a path; inferred from extension
    )    
    
    x=mesh.points[:,0]
    y=mesh.points[:,1]
    t=np.arange(0,68*1.467451833203625E-004,1.467451833203625E-004);

    u=mesh.point_data['flds1']#x
    v=mesh.point_data['flds2']#y
    p=mesh.point_data['flds3']#pressure
    N = x.shape[0]
    T = t.shape[0]
   # ——————————————————————————————————————————————————————————————————    

    x=x.flatten()[:,None]
    y=y.flatten()[:,None]
    t=t.flatten()[:,None]
    XX = np.tile(x, (1,T)) # N x T
    YY = np.tile(y, (1,T)) # N x T
    TT = np.tile(t, (N,1)) # N x T
    UU = np.tile(u, (1,T))
    VV = np.tile(v, (1,T))
    PP = np.tile(p, (1,T))
    
    x = XX.flatten()[:,None] # NT x 1
    y = YY.flatten()[:,None] # NT x 1
    t = TT.flatten()[:,None] # NT x 1
    
    u = UU.flatten()[:,None] # NT x 1
    v = VV.flatten()[:,None] # NT x 1
    p = PP.flatten()[:,None] # NT 
    
    ######################################################################
    ######################## Noiseless Data ###############################
    ##x####################################################################
    # Training Data /randomly taken within the range of all timestamp  
    idx = np.random.choice(N*T, N_train, replace=False)
    x_train = x[idx,:]
    y_train = y[idx,:]
    t_train = t[idx,:]
    u_train = u[idx,:]
    v_train = v[idx,:]


    # Apply the Training
    model = PhysicsInformedNN(x_train, y_train, t_train, u_train, v_train, layers)
    #The train iterations
    epoch=30000
    model.train(epoch)

### Testing Stage for noiseless data
Choose a certain time stamp's data for testing,

In [ ]:
    t=np.arange(0,68*1.467451833203625E-004,1.467451833203625E-004);
    TT = np.tile(t, (N,1))
    # Test Data /specificly taken at a certain time stamp, in this case 10
    snap = np.array([1])
    x_star = XX[:,snap]
    y_star = YY[:,snap]
    t_star = TT[:,snap]
    
    u_star = UU[:,snap]
    v_star = VV[:,snap]
    p_star = PP[:,snap]
    
    # Prediction
    u_pred, v_pred, p_pred = model.predict(x_star, y_star, t_star)
    lambda_1_value = model.sess.run(model.lambda_1)
    lambda_2_value = model.sess.run(model.lambda_2)
    
    # Error
    error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
    error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

    error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
    error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100
    print('Error u: %e' % (error_u))    
    print('Error v: %e' % (error_v))    
    print('Error p: %e' % (error_p))    
    print('Error l1: %.5f%%' % (error_lambda_1))                             
    print('Error l2: %.5f%%' % (error_lambda_2))  

### Implement the plotting function 

In [ ]:
def plot_solution(X_star, u_star, index):
    
    lb = X_star.min(0)
    ub = X_star.max(0)
    nn = 200
    x = np.linspace(lb[0], ub[0], nn)
    y = np.linspace(lb[1], ub[1], nn)
    X, Y = np.meshgrid(x,y)
    
    U_star = griddata(X_star, u_star.flatten(), (X, Y), method='cubic')
    
    plt.figure(index)
    plt.pcolor(X,Y,U_star, cmap = 'jet')
    plt.colorbar()
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/4
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

### Prediction plot of original test data

In [ ]:
    x1=mesh.points[:,0]
    y1=mesh.points[:,1]
    X_star=np.concatenate([x1.flatten()[:,None],y1.flatten()[:,None]],axis=1)
    plot_solution(X_star, u_star, 1)
    plot_solution(X_star, v_star, 2)
    plot_solution(X_star, p_star, 3)
    plot_solution(X_star, p_star, 4)
    plot_solution(X_star, p_star - p_pred, 5)

### Prediction plot of noiseless data

In [ ]:
    x1=mesh.points[:,0]
    y1=mesh.points[:,1]
    # Prediction plot
    X_star=np.concatenate([x1.flatten()[:,None],y1.flatten()[:,None]],axis=1)
    plot_solution(X_star, u_pred, 1)
    plot_solution(X_star, v_pred, 2)
    plot_solution(X_star, p_pred, 3)
    plot_solution(X_star, p_star, 4)
    plot_solution(X_star, p_star - p_pred, 5)

In [ ]:
    print(np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2))

### Training stage for noisy data

In [ ]:
    noise = 0.01        
    u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
    v_train = v_train + noise*np.std(v_train)*np.random.randn(v_train.shape[0], v_train.shape[1])    
    # Training
    model = PhysicsInformedNN(x_train, y_train, t_train, u_train, v_train, layers)
    model.train(10000)

### Testing stage for noisy data  

In [ ]:
    lambda_1_value_noisy = model.sess.run(model.lambda_1)
    lambda_2_value_noisy = model.sess.run(model.lambda_2)
    u_pred_noisy, v_pred_noisy, p_pred_noisy = model.predict(x_star, y_star, t_star)
    
    
    error_lambda_1_noisy = np.abs(lambda_1_value_noisy - 1.0)*100
    error_lambda_2_noisy = np.abs(lambda_2_value_noisy - 0.01)/0.01 * 100
    error_u_noisy = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    error_v_noisy = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
    error_p_noisy = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

    error_lambda_1 = np.abs(lambda_1_value - 1.0)*100
    error_lambda_2 = np.abs(lambda_2_value - 0.01)/0.01 * 100
    print('Error u_noisy: %e' % (error_u_noisy))    
    print('Error v_noisy: %e' % (error_v_noisy))    
    print('Error p_noisy: %e' % (error_p_noisy))       
    print('Error l1_noisy: %.5f%%' % (error_lambda_1_noisy))                             
    print('Error l2_noisy: %.5f%%' % (error_lambda_2_noisy))     


### Prediction plot of noiseless prediction 

In [ ]:
    plot_solution(X_star, u_pred_noisy, 1)
    plot_solution(X_star, v_pred_noisy, 2)
    plot_solution(X_star, p_pred_noisy, 3)
    plot_solution(X_star, p_star, 4)
    plot_solution(X_star, p_star - p_pred_noisy, 5)